### Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

'KERAS_BACKEND' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
##Updated to Keras 2.0
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using TensorFlow backend.
C:\Users\rajat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rajat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rajat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rajat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [4]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [5]:
# a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers.
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Training

In [7]:
epochs = 30
train_samples = 2048
validation_samples = 832

In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size,)
#About 60 seconds an epoch when using CPU


Epoch 1/30
64/64 [==============================] - 40s 618ms/step - loss: 0.7387 - accuracy: 0.5264 - val_loss: 0.6860 - val_accuracy: 0.5817
Epoch 2/30
64/64 [==============================] - 35s 540ms/step - loss: 0.6808 - accuracy: 0.5918 - val_loss: 0.6190 - val_accuracy: 0.6118
Epoch 3/30
64/64 [==============================] - 35s 549ms/step - loss: 0.6299 - accuracy: 0.6641 - val_loss: 0.6683 - val_accuracy: 0.6683
Epoch 4/30
64/64 [==============================] - 35s 544ms/step - loss: 0.5693 - accuracy: 0.7104 - val_loss: 0.5941 - val_accuracy: 0.7031
Epoch 5/30
64/64 [==============================] - 35s 543ms/step - loss: 0.5252 - accuracy: 0.7446 - val_loss: 0.6956 - val_accuracy: 0.6635
Epoch 6/30
64/64 [==============================] - 35s 547ms/step - loss: 0.4620 - accuracy: 0.7783 - val_loss: 0.5140 - val_accuracy: 0.7308
Epoch 7/30
64/64 [==============================] - 36s 557ms/step - loss: 0.4366 - accuracy: 0.7964 - val_loss: 0.5243 - val_accuracy: 0.720

In [9]:
model.save_weights('models/basic_cnn_30_epochs.h5')

In [11]:
model.save_weights('models_trained/basic_cnn_30_epochs.h5')
model.load_weights('models_trained/basic_cnn_30_epochs.h5')

If your model successfully runs at one epoch, go back and it for 30 epochs by changing epochs above.  I was able to get to an val_acc of 0.71 at 30 epochs.
A copy of a pretrained network is available in the pretrained folder.

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
model.evaluate_generator(validation_generator, validation_samples)

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reach ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs.**